# 네이버 뉴스 연구 마스터 테이블 만들기

In [ ]:
import pandas as pd
import polars as pl
import json
from tqdm import tqdm
from json import JSONDecodeError
import pickle

In [ ]:
df = pd.read_excel('./final_simple_stat.xlsx')
df = df.dropna(subset=['inclusion'])
df = df.loc[df['inclusion']==1.0]
df

,idx,title,inclusion,date,n_reply,url
0,0,"文 ""원전, 60년 주력 전원…신한울 1·2호기 이른시간 내 가동""(종합2보)",1.0,2022.02.25. 오후 8:13,999,https://n.news.naver.com/mnews/article/001/001...
2,2,원전 확대·4대강사업 계승…MB 시절 사회갈등 되풀이하나,1.0,2022.03.15. 오전 5:01,984,https://n.news.naver.com/mnews/article/028/000...
3,3,與 “원자력 소외된 아픔 드린 것 사과”… 원전 전문가도 영입,1.0,2022.01.27. 오후 4:21,979,https://n.news.naver.com/mnews/article/023/000...
5,5,"탈원전에 3~5년 늦어진 원전… 文대통령, 이제와서 “조기가동”",1.0,2022.02.25. 오후 6:37,948,https://n.news.naver.com/mnews/article/366/000...
7,7,헝가리 이어 체코 정상도…문대통령 앞에서 연이은 원전발언,1.0,2021.11.04. 오후 11:13,927,https://n.news.naver.com/mnews/article/001/001...
...,...,...,...,...,...,...
501,501,이제야 '탈원전 족쇄' 푸는 정부…원전 수명 다시 늘린다,1.0,2022.04.06. 오후 5:55,101,https://n.news.naver.com/mnews/article/009/000...
503,503,"이재명 ""2040년 내연차 판매중단""…탄소세도 공식화",1.0,2021.08.26. 오전 10:59,100,https://n.news.naver.com/mnews/article/001/001...
504,504,"文대통령 ""韓 수소경제 빠르게 성장...세계최고 수준 기술""",1.0,2021.11.11. 오후 12:35,100,https://n.news.naver.com/mnews/article/008/000...
505,505,"""대선후 전기료 더 오를듯""…한전 올해 10조 적자 불가피",1.0,2022.02.06. 오후 5:25,100,https://n.news.naver.com/mnews/article/009/000...


In [ ]:
# 연구 대상 json 파일
file_list = df['idx'].tolist()

In [ ]:
del df

In [ ]:
# 댓글 제외 json
json_list = []
for file in tqdm(file_list):
    try:
        with open(f'./news_json_files/{file}.json',encoding='utf-8-sig') as f:
            json_object = json.load(f)
        del json_object['dic_reply']
        json_object['idx'] = file
        json_list.append(json_object)
    except JSONDecodeError:
        print(file)
    except FileNotFoundError:
        print(f'{file} is not found')

In [ ]:
del json_object

In [ ]:
news = pd.DataFrame(json_list)
news

In [ ]:
news.to_pickle('./news_df.pkl')

In [ ]:
del json_list, news

In [ ]:
# with open('./json_reply_list.pkl','wb') as f:
#     pickle.dump(json_reply_list,f)

### 기사 댓글 (댓글 + 대댓글) 전처리

In [ ]:
# 댓글 + 대댓글 json
json_reply_list = []
for file in tqdm(file_list):
    try:
        with open(f'./news_json_files/{file}.json',encoding='utf-8-sig') as f:
            json_object = json.load(f)
        temp = {}
        temp['idx'] = file
        temp['reply'] = [r['reply_text'] for r in json_object['dic_reply'] if r['reply_text']!=''] + [s['son_reply_text'] for r in json_object['dic_reply'] for s in r['dic_son_reply'] if s['son_reply_text']!='']
        # json_reply_list.append({key:json_object[key] for key in ['idx','reply']})
        json_reply_list.append(temp)
    except JSONDecodeError:
        print(file)
    except FileNotFoundError:
        print(f'{file} is not found')

In [ ]:
pd.DataFrame(json_reply_list).to_pickle('./reply_df.pkl')

### 유저별 과거 이력 정보

In [ ]:
#  'user_id': '1QScg',
#  'nickname': 'joon****',
#  'user_real_nickname': '이리오너라',
#  'start_date': '2018.01.02.',
#  'sum_reply': '1', # 댓글
#  'sum_son_reply': '3', # 답글
#  'sum_emotion': '17', # 받은 공감 (좋아요/싫어요 합)
#  'recent_n_reply': '0',
#  'recent_n_deleted': '0',
#  'recent_n_emotion': '0',
#  'recent_rate_emotion': '0%',
#  'recent_rate_self_delete': '0%'
# [r['dic_history'] for r in df['dic_reply'][0]][0]

In [ ]:
len(file_list)

In [ ]:
# 댓글 유저 히스토리 json
json_hist_list = []
for file in tqdm(file_list):
    try:
        with open(f'./news_json_files/{file}.json',encoding='utf-8-sig') as f:
            json_object = json.load(f)
        temp = {}
        temp['idx'] = file
        temp['dic_history'] = [r['dic_history'] for r in json_object['dic_reply']]
        temp['dic_history'] = [{key:d[key] for key in list(d.keys())[:-1]} for d in temp['dic_history'] if d!='']
        # json_reply_list.append({key:json_object[key] for key in ['idx','reply']})
        json_hist_list.append(temp)
    except JSONDecodeError:
        print(file)
    except FileNotFoundError:
        print(f'{file} is not found')

In [ ]:
pd.DataFrame(json_hist_list).to_pickle('./reply_user_df.pkl')

### 유저별 과거 댓글 정보

In [ ]:
# 댓글 유저 과거 댓글 json
json_hist_reply_list = []
for file in tqdm(file_list):
    try:
        with open(f'./news_json_files/{file}.json',encoding='utf-8-sig') as f:
            json_object = json.load(f)
        # temp = {}
        # temp['idx'] = file
        dic_history = [r['dic_history'] for r in json_object['dic_reply']]
        # temp['user_id'] = [{d['user_id']} for d in dic_history if d!='']
        # temp['dic_reply_history'] = [{d['dic_reply_history']} for d in dic_history if d!='']
        json_hist_reply_list.append([{key:d[key] for key in ['user_id','dic_reply_history']} for d in dic_history if d!=''])
        # json_hist_reply_list.append(temp)
    except JSONDecodeError:
        print(file)
    except FileNotFoundError:
        print(f'{file} is not found')

### 감정/혐오 분류

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.3 MB/s eta 0:00:00


In [11]:
#혐오 발언에 대한 분류
from transformers import TextClassificationPipeline, BertForSequenceClassification, AutoTokenizer
model_name = 'sgunderscore/hatescore-korean-hate-speech'
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
pipe = TextClassificationPipeline(
        model = model,
        tokenizer = tokenizer,
        # device = 0, # gpu: 0
        top_k=None,
        function_to_apply = 'sigmoid')

In [14]:
for result in pipe("")[0]:
        print(result)

{'label': '인종/국적', 'score': 0.9042615294456482}
{'label': 'None', 'score': 0.1099800392985344}
{'label': '종교', 'score': 0.03681234270334244}
{'label': '기타 혐오', 'score': 0.029042314738035202}
{'label': '지역', 'score': 0.018468299880623817}
{'label': '단순 악플', 'score': 0.015187964774668217}
{'label': '여성/가족', 'score': 0.013147241435945034}
{'label': '성소수자', 'score': 0.012589571997523308}
{'label': '연령', 'score': 0.012162875384092331}
{'label': '남성', 'score': 0.010752644389867783}


In [16]:
#댓글 감정 분석
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

model_name = "searle-j/kote_for_easygoing_people"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

pipe = TextClassificationPipeline(
        model=model,
        tokenizer=tokenizer,
        device=-1, # gpu number, -1 if cpu used
        top_k=None,
        function_to_apply='sigmoid'
    )

In [17]:
for output in pipe("""장담하는데 너는 곱게 죽지 못한다.!! 왜? 국민이 이를 갈고 있거든..!!""")[0]:
    if output["score"]>0.4:
        print(output)

{'label': '화남/분노', 'score': 0.9090842008590698}
{'label': '증오/혐오', 'score': 0.771856427192688}
{'label': '비장함', 'score': 0.5468642711639404}
{'label': '불평/불만', 'score': 0.4599364399909973}


In [18]:
for output in pipe("""저질들이 나라를 잠식하면 오늘날 이모양 이꼴이 되죠. 국민이 속아서 그렇게 된거이니 이제부터 절대 속지 맙시다.! 저들은 이제 절대 사회에서 활동 못하게 해야 합니다.!!""")[0]:
    if output["score"]>0.4:
        print(output)

{'label': '화남/분노', 'score': 0.9827980399131775}
{'label': '불평/불만', 'score': 0.9427919387817383}
{'label': '증오/혐오', 'score': 0.9233428835868835}
{'label': '의심/불신', 'score': 0.8942246437072754}
{'label': '안타까움/실망', 'score': 0.7989993691444397}
{'label': '비장함', 'score': 0.7964633703231812}
{'label': '짜증', 'score': 0.7653007507324219}
{'label': '한심함', 'score': 0.5697562098503113}
{'label': '불안/걱정', 'score': 0.4971129596233368}
{'label': '지긋지긋', 'score': 0.461895614862442}
{'label': '역겨움/징그러움', 'score': 0.45308905839920044}
